In [3]:
from data_loader import load_data

greedy → simulated annealing → tabu search

This solution combines three optimization techniques: Greedy, Simulated Annealing (SA), and Tabu Search. First, a quick initial solution is generated using a greedy approach, which assigns each customer to the nearest depot that can accommodate their demand. Next, Simulated Annealing is applied to iteratively improve the solution by exploring neighboring solutions with a probability of accepting worse solutions, which decreases over time to avoid local minima. Finally, Tabu Search further refines the solution by maintaining a tabu list that prevents revisiting recently explored solutions, encouraging exploration of new, potentially better solutions. Together, these techniques balance exploration and exploitation, leading to an optimized solution for the facility location problem.

In [11]:
import random
import numpy as np

def total_cost(solution, setup_costs, costs):
    depot_cost = sum(setup_costs[d] for d in solution['open_depots'])
    transport_cost = sum(costs[i][solution['assignment'][i]] for i in range(len(solution['assignment'])))
    return depot_cost + transport_cost

def delta_cost(old_solution, new_solution, setup_costs, costs):
    old_open = set(old_solution['open_depots'])
    new_open = set(new_solution['open_depots'])
    depot_diff = sum(setup_costs[d] for d in new_open - old_open) - sum(setup_costs[d] for d in old_open - new_open)
    assignment_old = old_solution['assignment']
    assignment_new = new_solution['assignment']
    transport_diff = sum(
        costs[i][assignment_new[i]] - costs[i][assignment_old[i]]
        for i in range(len(assignment_old)) if assignment_new[i] != assignment_old[i]
    )
    return depot_diff + transport_diff

def is_feasible(assignment, depot_capacities, customer_demands, n_depots):
    depot_loads = [0] * n_depots
    for i, d in enumerate(assignment):
        depot_loads[d] += customer_demands[i]
    return all(depot_loads[i] <= depot_capacities[i] for i in range(n_depots))

def initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs):
    assignment = [-1] * n_customers
    remaining_capacity = depot_capacities[:]
    for i in range(n_customers):
        sorted_depots = sorted(range(n_depots), key=lambda d: costs[i][d])
        for d in sorted_depots:
            if remaining_capacity[d] >= customer_demands[i]:
                assignment[i] = d
                remaining_capacity[d] -= customer_demands[i]
                break
    open_depots = list(set(assignment))
    return {'assignment': assignment, 'open_depots': open_depots}

def get_neighbors_limited(solution, n_depots, depot_capacities, customer_demands, k=3):
    neighbors = []
    assignment = solution['assignment']
    for _ in range(k):
        i = random.randint(0, len(assignment)-1)
        current_d = assignment[i]
        for d in range(n_depots):
            if d != current_d:
                new_assignment = assignment[:]
                new_assignment[i] = d
                if is_feasible(new_assignment, depot_capacities, customer_demands, n_depots):
                    open_depots = list(set(new_assignment))
                    neighbors.append({'assignment': new_assignment, 'open_depots': open_depots})
    return neighbors

def simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                        initial_temp=1000, cooling_rate=0.95, max_iter=100, neighbor_k=5):
    current = initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
    best = current
    temp = initial_temp
    for _ in range(max_iter):
        neighbors = get_neighbors_limited(current, n_depots, depot_capacities, customer_demands, k=neighbor_k)
        if not neighbors:
            break
        candidate = random.choice(neighbors)
        cost_diff = total_cost(candidate, setup_costs, costs) - total_cost(current, setup_costs, costs)
        if cost_diff < 0 or random.random() < np.exp(-cost_diff / temp):
            current = candidate
            if total_cost(current, setup_costs, costs) < total_cost(best, setup_costs, costs):
                best = current
        temp *= cooling_rate
    return best

def tabu_search_optimized(initial_sol, n_depots, depot_capacities, customer_demands,
                          setup_costs, costs, max_iter=50, tabu_tenure=5, neighbor_k=10, runs=3):
    best_global = None
    for _ in range(runs):
        current = initial_sol
        best = current
        best_cost = total_cost(best, setup_costs, costs)
        tabu_list = []
        for _ in range(max_iter):
            neighbors = get_neighbors_limited(current, n_depots, depot_capacities, customer_demands, k=neighbor_k)
            neighbors = [n for n in neighbors if n['assignment'] not in tabu_list]
            if not neighbors:
                break
            candidate = min(neighbors, key=lambda s: delta_cost(current, s, setup_costs, costs))
            cost_candidate = total_cost(candidate, setup_costs, costs)
            if cost_candidate < best_cost:
                best = candidate
                best_cost = cost_candidate
            tabu_list.append(candidate['assignment'])
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
            current = candidate
        if best_global is None or total_cost(best, setup_costs, costs) < total_cost(best_global, setup_costs, costs):
            best_global = best
    return best_global


In [14]:
filename = 'Dataset/wl_25'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

greedy_init = initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
sa_result = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)
final_solution = tabu_search_optimized(sa_result, n_depots, depot_capacities, customer_demands,
                                       setup_costs, costs, max_iter=40, tabu_tenure=4, neighbor_k=3, runs=2)
print("Total cost : ",total_cost(final_solution, setup_costs, costs))
print("Open depots : ",final_solution['open_depots'])
print("Assignment : ",final_solution['assignment'])

Total cost :  813092.9875
Open depots :  [0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 15, 16, 19, 20, 22, 23, 24]
Assignment :  [7, 6, 0, 24, 20, 0, 1, 5, 20, 7, 3, 4, 5, 0, 6, 7, 19, 8, 20, 6, 20, 15, 4, 0, 11, 4, 12, 4, 19, 0, 15, 10, 15, 16, 11, 11, 24, 5, 23, 24, 19, 20, 7, 6, 22, 23, 23, 6, 24, 11]


In [15]:
filename = 'Dataset/wl_50'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

greedy_init = initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
sa_result = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)
final_solution = tabu_search_optimized(sa_result, n_depots, depot_capacities, customer_demands,
                                       setup_costs, costs, max_iter=40, tabu_tenure=4, neighbor_k=3, runs=2)
print("Total cost : ",total_cost(final_solution, setup_costs, costs))
print("Open depots : ",final_solution['open_depots'])
print("Assignment : ",final_solution['assignment'])

Total cost :  963804.15
Open depots :  [4, 5, 10, 11, 12, 13, 18, 19, 21, 22, 25, 26, 27, 29, 33, 35, 36, 37, 40, 41, 44, 45]
Assignment :  [19, 40, 5, 36, 4, 5, 36, 12, 4, 35, 10, 11, 12, 25, 21, 35, 10, 22, 18, 19, 41, 21, 22, 13, 40, 25, 26, 27, 27, 29, 5, 27, 5, 33, 40, 35, 36, 37, 45, 37, 40, 41, 4, 21, 44, 45, 4, 19, 37, 40]


In [16]:
filename = 'Dataset/wl_200'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

greedy_init = initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
sa_result = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)
final_solution = tabu_search_optimized(sa_result, n_depots, depot_capacities, customer_demands,
                                       setup_costs, costs, max_iter=40, tabu_tenure=4, neighbor_k=3, runs=2)
print("Total cost : ",total_cost(final_solution, setup_costs, costs))
print("Open depots : ",final_solution['open_depots'])
print("Assignment : ",final_solution['assignment'])

Total cost :  36262.55734000001
Open depots :  [0, 2, 3, 4, 5, 8, 10, 12, 13, 14, 19, 20, 21, 24, 25, 28, 31, 32, 34, 36, 37, 38, 43, 44, 45, 47, 48, 49, 51, 52, 53, 61, 63, 64, 66, 70, 71, 72, 75, 79, 80, 81, 83, 84, 85, 87, 92, 93, 95, 100, 102, 106, 107, 109, 110, 111, 113, 119, 121, 125, 126, 127, 128, 129, 130, 133, 136, 137, 138, 139, 142, 143, 144, 147, 148, 149, 150, 154, 157, 158, 162, 165, 166, 167, 168, 169, 170, 173, 176, 178, 183, 184, 185, 188, 194, 196, 197, 198]
Assignment :  [197, 176, 84, 125, 173, 53, 185, 93, 113, 3, 142, 198, 111, 130, 162, 47, 66, 21, 148, 2, 197, 137, 71, 142, 32, 48, 44, 95, 106, 34, 184, 178, 48, 93, 52, 102, 85, 20, 70, 10, 79, 149, 194, 147, 166, 14, 19, 63, 133, 126, 37, 125, 158, 32, 183, 129, 13, 106, 51, 53, 154, 162, 102, 53, 85, 188, 165, 121, 70, 38, 166, 49, 87, 64, 147, 25, 144, 5, 20, 81, 198, 138, 14, 178, 53, 61, 109, 24, 150, 2, 45, 24, 130, 184, 128, 102, 169, 176, 12, 121, 34, 165, 38, 168, 53, 25, 8, 13, 198, 21, 93, 49, 47, 9

In [17]:
filename = 'Dataset/wl_300'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

greedy_init = initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
sa_result = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)
final_solution = tabu_search_optimized(sa_result, n_depots, depot_capacities, customer_demands,
                                       setup_costs, costs, max_iter=40, tabu_tenure=4, neighbor_k=3, runs=2)
print("Total cost : ",total_cost(final_solution, setup_costs, costs))
print("Open depots : ",final_solution['open_depots'])
print("Assignment : ",final_solution['assignment'])

Total cost :  87000.74234
Open depots :  [0, 1, 2, 3, 8, 11, 12, 13, 18, 19, 20, 22, 23, 24, 26, 27, 28, 30, 31, 32, 34, 37, 38, 40, 42, 45, 46, 47, 48, 49, 51, 52, 53, 54, 56, 58, 59, 61, 62, 65, 69, 74, 77, 82, 83, 84, 92, 93, 96, 97, 100, 101, 102, 103, 106, 107, 110, 113, 115, 117, 118, 119, 121, 122, 124, 125, 126, 129, 132, 134, 135, 138, 140, 141, 143, 146, 147, 148, 149, 156, 160, 161, 163, 164, 166, 169, 172, 173, 178, 181, 184, 185, 186, 187, 188, 191, 199, 200, 204, 206, 209, 210, 211, 212, 215, 217, 219, 220, 221, 223, 225, 226, 231, 232, 234, 235, 236, 237, 238, 239, 240, 241, 244, 248, 251, 252, 253, 256, 258, 264, 265, 266, 268, 272, 273, 275, 276, 277, 278, 279, 281, 282, 284, 286, 287, 288, 289, 290, 292, 294, 296, 297, 299]
Assignment :  [258, 53, 30, 256, 125, 135, 225, 275, 160, 100, 221, 275, 240, 235, 156, 278, 161, 13, 200, 118, 238, 268, 138, 226, 103, 169, 149, 147, 268, 45, 220, 51, 236, 172, 237, 200, 1, 132, 284, 212, 22, 37, 289, 12, 219, 32, 96, 48, 83, 11

In [18]:
filename = 'Dataset/wl_500'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

greedy_init = initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
sa_result = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)
final_solution = tabu_search_optimized(sa_result, n_depots, depot_capacities, customer_demands,
                                       setup_costs, costs, max_iter=40, tabu_tenure=4, neighbor_k=3, runs=2)
print("Total cost : ",total_cost(final_solution, setup_costs, costs))
print("Open depots : ",final_solution['open_depots'])
print("Assignment : ",final_solution['assignment'])

Total cost :  88992.01753
Open depots :  [0, 2, 4, 6, 7, 8, 9, 12, 13, 14, 17, 19, 21, 22, 23, 25, 26, 27, 29, 31, 33, 37, 38, 51, 52, 53, 54, 56, 58, 59, 60, 64, 65, 66, 67, 69, 72, 73, 74, 75, 76, 78, 79, 82, 84, 87, 88, 90, 91, 92, 94, 97, 98, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 118, 120, 121, 122, 123, 126, 127, 128, 129, 130, 131, 133, 134, 135, 137, 138, 140, 141, 144, 147, 148, 149, 150, 154, 158, 161, 162, 163, 164, 165, 166, 168, 171, 173, 174, 175, 176, 180, 182, 183, 184, 185, 186, 187, 192, 195, 196, 198, 199, 202, 206, 208, 210, 213, 215, 216, 217, 220, 221, 225, 231, 244, 247, 248, 251, 252, 253, 256, 257, 258, 260, 261, 262, 263, 265, 266, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 285, 286, 290, 293, 297, 304, 305, 308, 309, 312, 318, 319, 320, 321, 322, 324, 325, 330, 331, 332, 333, 334, 335, 336, 337, 338, 341, 342, 344, 345, 348, 350, 351, 354, 355, 356, 359, 360, 363, 364, 365, 367, 368, 369, 370, 371, 372, 375